# INST326 — Week 10 Exercises: Data Models, Properties, and Special Methods (Library Management)

**Focus (Week 10 only):** Python data model & class design refinements — `@property` (get/set/delete) with validation, dataclass options (equality, ordering, `field` options, `__post_init__`), special methods (`__str__`, `__repr__`, `__len__`, `__iter__`, `__contains__`, `__eq__`, ordering), and lightweight container patterns. Optional: `functools.total_ordering` and `dataclasses.replace`.

**Out of scope (Week 11+):** multiple inheritance/mixins, advanced design patterns, descriptors beyond `@property`, metaclasses, concurrency, ORMs, complex type-system features, network I/O, or frameworks.


### Starter Scaffold (Week-10-safe)

Below is minimal starter code from prior weeks, lightly adapted for Week 10. You may extend it while staying within the Week 10 scope.


In [ ]:
from __future__ import annotations
from dataclasses import dataclass, field, replace
from datetime import datetime, timedelta
from typing import Dict, Iterable, Iterator, Optional, List
from functools import total_ordering

# --- Exceptions (basic) ---
class LibraryError(Exception): ...
class DuplicateBookError(LibraryError): ...
class OverdueLoanError(LibraryError): ...

@total_ordering
@dataclass(order=False)
class Book:
    isbn: str
    title: str
    _copies: int = field(default=1, repr=False, compare=False)

    # Comparable by title then ISBN (Week 10: ordering/special methods)
    def __lt__(self, other: "Book") -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return (self.title, self.isbn) < (other.title, other.isbn)

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return self.isbn == other.isbn  # equality by identity key

    def __repr__(self) -> str:
        return f"Book(isbn={self.isbn!r}, title={self.title!r}, copies={self._copies})"

    def __str__(self) -> str:
        return f"{self.title} [{self.isbn}]"

    # Week 10: properties for validation
    @property
    def copies(self) -> int:
        return self._copies

    @copies.setter
    def copies(self, value: int) -> None:
        if not isinstance(value, int):
            raise TypeError("copies must be int")
        if value < 0:
            raise ValueError("copies must be non-negative")
        self._copies = value

    @copies.deleter
    def copies(self) -> None:
        # Reset to zero (logical 'remove from circulation')
        self._copies = 0

@dataclass
class Member:
    member_id: str
    email: str

@dataclass
class Loan:
    isbn: str
    member_id: str
    due_date: datetime
    returned: bool = False

    def mark_returned(self) -> None:
        self.returned = True

class Catalog(Iterable[Book]):
    """Lightweight container with special methods (Week 10)."""
    def __init__(self):
        self._books: Dict[str, Book] = {}

    # Container-like dunder methods
    def __len__(self) -> int:
        return len(self._books)

    def __iter__(self) -> Iterator[Book]:
        # Iterate in title order for determinism
        return iter(sorted(self._books.values()))

    def __contains__(self, isbn: object) -> bool:
        # Allow 'in' checks by ISBN or Book
        if isinstance(isbn, Book):
            return isbn.isbn in self._books
        return isinstance(isbn, str) and (isbn in self._books)

    def __getitem__(self, isbn: str) -> Book:
        return self._books[isbn]

    # API
    def add_book(self, book: Book) -> None:
        if book.isbn in self._books:
            raise DuplicateBookError(f"ISBN exists: {book.isbn}")
        self._books[book.isbn] = book

    def get_book(self, isbn: str) -> Optional[Book]:
        return self._books.get(isbn)

    def remove_book(self, isbn: str) -> Optional[Book]:
        return self._books.pop(isbn, None)

class LoanDesk:
    def __init__(self, catalog: Catalog):
        self.catalog = catalog
        self.loans: List[Loan] = []

    def checkout(self, member: Member, book: Book) -> Loan:
        if book.copies <= 0:
            raise LibraryError("no available copies")
        book.copies -= 1
        due = datetime.now() + timedelta(days=14)
        loan = Loan(isbn=book.isbn, member_id=member.member_id, due_date=due)
        self.loans.append(loan)
        return loan

    def checkin(self, loan: Loan) -> None:
        if not loan.returned:
            b = self.catalog.get_book(loan.isbn)
            if b:
                b.copies += 1
            loan.mark_returned()


## 1) Property validation

Add an `@property` named `title` to `Book` that strips whitespace on set and raises `ValueError` if empty after stripping. Keep the existing attribute name (override dataclass behavior carefully).

In [ ]:
# Your code here
# Hint: define a private _title and redirect in __post_init__ if needed.
@total_ordering
@dataclass(order=False)
class Book:
    isbn: str
    title: str  # This will be overridden by @property
    _copies: int = field(default=1, repr=False, compare=False)
    author: str = field(default="Unknown")
    _title: str = field(default="", init=False, repr=False, compare=False)

    def __post_init__(self):
        if not self.isbn or len(self.isbn) < 10:
            raise ValueError("ISBN must be at least 10 characters")
        self.title = self.title

    def __lt__(self, other: "Book") -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return (self._title, self.isbn) < (other._title, other.isbn)

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return self.isbn == other.isbn  

    def __repr__(self) -> str:
        return f"Book(isbn={self.isbn!r}, title={self._title!r}, copies={self._copies})"

    def __str__(self) -> str:
        return f"{self._title} by {self.author} [{self.isbn}]"

    def __hash__(self) -> int:
        return hash(self.isbn)


    @property
    def copies(self) -> int:
        return self._copies

    @copies.setter
    def copies(self, value: int) -> None:
        if not isinstance(value, int):
            raise TypeError("copies must be int")
        if value < 0:
            raise ValueError("copies must be non-negative")
        self._copies = value

    @copies.deleter
    def copies(self) -> None:
        self._copies = 0

    @property
    def is_available(self) -> bool:
        return self._copies > 0

    @property
    def title(self) -> str:
        return self._title

    @title.setter
    def title(self, value: str) -> None:
        if not isinstance(value, str):
            raise TypeError("title must be a string")
        stripped = value.strip()
        if not stripped:
            raise ValueError("Title cannot be empty after stripping whitespace")
        self._title = stripped

## 2) Read-only computed property

Add a read-only property `label` on `Book` that returns `f"{title} [{isbn}]"`. Attempting to set `label` should raise `AttributeError`. Demonstrate usage.

In [ ]:
# Your code here
@property
def label(self) -> str:
    return f"{self.title} [{self.isbn}]"


book = Book(isbn="978-0134685991", title="Effective Python", author="Brett Slatkin")

print(book.label)  

try:
    book.label = "New Label"
except AttributeError as e:
    print(f"Error: {e}")  

book.title = "Effective Python (2nd Edition)"
print(book.label)  

## 3) `__post_init__` data normalization

Use `__post_init__` on `Book` to normalize ISBN by stripping hyphens/spaces and uppercasing the title. Ensure validation runs on initial values too.

In [ ]:
# Your code here
@total_ordering
@dataclass(order=False)
class Book:
    isbn: str
    title: str 
    _copies: int = field(default=1, repr=False, compare=False)
    author: str = field(default="Unknown")
    _title: str = field(default="", init=False, repr=False, compare=False)

    def __post_init__(self):
        self.isbn = self.isbn.replace("-", "").replace(" ", "")
        
        if not self.isbn or len(self.isbn) < 10:
            raise ValueError("ISBN must be at least 10 characters after normalization")
        
        self.title = self.title
        
        self._title = self._title.upper()


    @property
    def title(self) -> str:
        return self._title

    @title.setter
    def title(self, value: str) -> None:
        if not isinstance(value, str):
            raise TypeError("title must be a string")
        stripped = value.strip()
        if not stripped:
            raise ValueError("Title cannot be empty after stripping whitespace")
        self._title = stripped

## 4) `dataclasses.field` options

Modify `Book` so `_copies` is excluded from comparisons (`compare=False`) and included in `__repr__` via a custom `__repr__`. Explain briefly in a comment why we avoid comparing by copies.

In [ ]:
# Your code here

@total_ordering
@dataclass(order=False)
class Book:
    isbn: str
    title: str  
    _copies: int = field(default=1, repr=False, compare=False)
    author: str = field(default="Unknown")
    _title: str = field(default="", init=False, repr=False, compare=False)


    def __post_init__(self):
        self.isbn = self.isbn.replace("-", "").replace(" ", "")

        if not self.isbn or len(self.isbn) < 10:
            raise ValueError("ISBN must be at least 10 characters after normalization")
        
        self.title = self.title
        
        self._title = self._title.upper()

    def __repr__(self) -> str:
        return f"Book(isbn={self.isbn!r}, title={self._title!r}, author={self.author!r}, copies={self._copies})"

    def __str__(self) -> str:
        return f"{self._title} by {self.author} [{self.isbn}]"

    def __lt__(self, other: "Book") -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return (self._title, self.isbn) < (other._title, other.isbn)

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return self.isbn == other.isbn  

    def __hash__(self) -> int:
        return hash(self.isbn)

    @property
    def copies(self) -> int:
        return self._copies

    @copies.setter
    def copies(self, value: int) -> None:
        if not isinstance(value, int):
            raise TypeError("copies must be int")
        if value < 0:
            raise ValueError("copies must be non-negative")
        self._copies = value

    @property
    def title(self) -> str:
        return self._title

    @title.setter
    def title(self, value: str) -> None:
        if not isinstance(value, str):
            raise TypeError("title must be a string")
        stripped = value.strip()
        if not stripped:
            raise ValueError("Title cannot be empty after stripping whitespace")
        self._title = stripped

    @property
    def label(self) -> str:
        return f"{self.title} [{self.isbn}]"

## 5) Ordering with `total_ordering`

Given `__eq__` (by ISBN) and `__lt__` (by title, then ISBN), verify that sorting a mixed list of `Book` objects orders by title. Show a quick sort demo.

In [ ]:
# Your code here
books = [
    Book(isbn="978-0134685991", title="Effective Python", _copies=3),
    Book(isbn="978-0135957059", title="A Philosophy of Software Design", _copies=2),
    Book(isbn="978-1491946008", title="Fluent Python", _copies=5),
    Book(isbn="978-0132350884", title="Clean Code", _copies=4),
    Book(isbn="978-0201633610", title="Design Patterns", _copies=1),
    Book(isbn="978-0134685991", title="Effective Python", _copies=10),  # Duplicate ISBN, different copies
    Book(isbn="978-0596007126", title="Beautiful Code", _copies=2),
]

print("Original order:")
for book in books:
    print(f"  {book.title} [{book.isbn}] - {book.copies} copies")

print("\n" + "="*60 + "\n")

sorted_books = sorted(books)

print("After sorting (by title, then ISBN):")
for book in sorted_books:
    print(f"  {book.title} [{book.isbn}] - {book.copies} copies")

print("\n" + "="*60 + "\n")

print("Verification:")
print(f"First book: {sorted_books[0].title}")  
print(f"Last book: {sorted_books[-1].title}")   
print(f"\nOriginal list length: {len(books)}")  
print(f"Unique books (by ISBN in set): {len(set(books))}")  

print("\nTitle-based ordering examples:")
print(f"'A PHILOSOPHY OF SOFTWARE DESIGN' < 'BEAUTIFUL CODE': {sorted_books[0] < sorted_books[1]}")
print(f"'CLEAN CODE' < 'DESIGN PATTERNS': {sorted_books[2] < sorted_books[3]}")
print(f"'EFFECTIVE PYTHON' < 'FLUENT PYTHON': {sorted_books[4] < sorted_books[5]}")

book_a = Book(isbn="111-1111111", title="Python Guide", _copies=1)
book_b = Book(isbn="222-2222222", title="Python Guide", _copies=5)
print(f"\nSame title, different ISBN:")
print(f"  Book A: {book_a.label}")
print(f"  Book B: {book_b.label}")
print(f"  A < B (ISBN tiebreaker): {book_a < book_b}")  # True (111... < 222...)
print(f"  Sorted order: {sorted([book_b, book_a])[0].isbn}")  # Should be book_a first

## 6) Container protocol methods

Implement `__delitem__(self, isbn: str)` on `Catalog` to remove a book or raise `KeyError` if missing. Show that `len(catalog)` updates accordingly.

In [ ]:
# Your code here

class Catalog(Iterable[Book]):
    def __init__(self):
        self._books: Dict[str, Book] = {}


    def __len__(self) -> int:
        return len(self._books)

    def __iter__(self) -> Iterator[Book]:
        return iter(sorted(self._books.values()))

    def __contains__(self, isbn: object) -> bool:
        if isinstance(isbn, Book):
            return isbn.isbn in self._books
        return isinstance(isbn, str) and (isbn in self._books)

    def __getitem__(self, isbn: str) -> Book:
        return self._books[isbn]

    def __delitem__(self, isbn: str) -> None:
        if isbn not in self._books:
            raise KeyError(f"Book with ISBN {isbn} not found in catalog")
        del self._books[isbn]

    def __str__(self) -> str:
        return f"Catalog with {len(self)} book(s)"

    def __repr__(self) -> str:
        return f"Catalog({list(self._books.keys())})"

    def add_book(self, book: Book) -> None:
        if book.isbn in self._books:
            raise DuplicateBookError(f"ISBN exists: {book.isbn}")
        self._books[book.isbn] = book

    def get_book(self, isbn: str) -> Optional[Book]:
        return self._books.get(isbn)

    def remove_book(self, isbn: str) -> Optional[Book]:
        return self._books.pop(isbn, None)

    def search_by_title(self, title: str) -> List[Book]:
        title_lower = title.lower()
        return [b for b in self if title_lower in b._title.lower()]


## 7) Slicing/lookup convenience

Implement `find_by_title_prefix(self, prefix: str) -> list[Book]` on `Catalog` that returns books whose title starts with the prefix (case-insensitive).

In [ ]:
# Your code here

class Catalog(Iterable[Book]):
    def __init__(self):
        self._books: Dict[str, Book] = {}

    def __len__(self) -> int:
        return len(self._books)

    def __iter__(self) -> Iterator[Book]:
        return iter(sorted(self._books.values()))

    def __contains__(self, isbn: object) -> bool:
        if isinstance(isbn, Book):
            return isbn.isbn in self._books
        return isinstance(isbn, str) and (isbn in self._books)

    def __getitem__(self, isbn: str) -> Book:
        return self._books[isbn]

    def __delitem__(self, isbn: str) -> None:
        if isbn not in self._books:
            raise KeyError(f"Book with ISBN {isbn} not found in catalog")
        del self._books[isbn]

    def __str__(self) -> str:
        return f"Catalog with {len(self)} book(s)"

    def __repr__(self) -> str:
        return f"Catalog({list(self._books.keys())})"

   
    def add_book(self, book: Book) -> None:
        if book.isbn in self._books:
            raise DuplicateBookError(f"ISBN exists: {book.isbn}")
        self._books[book.isbn] = book

    def get_book(self, isbn: str) -> Optional[Book]:
        return self._books.get(isbn)

    def remove_book(self, isbn: str) -> Optional[Book]:
        return self._books.pop(isbn, None)

    def search_by_title(self, title: str) -> List[Book]:
        title_lower = title.lower()
        return [b for b in self if title_lower in b._title.lower()]

    def find_by_title_prefix(self, prefix: str) -> List[Book]:
        prefix_lower = prefix.lower()
        return [book for book in self if book._title.lower().startswith(prefix_lower)]


## 8) Immutability via `replace`

Using `dataclasses.replace`, show how to create a new `Book` with a different title while leaving the original unchanged. Explain in a comment when immutability is helpful.

In [ ]:
# Your code here

from dataclasses import replace

original_book = Book(
    isbn="978-0134685991",
    title="Effective Python",
    author="Brett Slatkin",
    _copies=5
)

print("Original book:")
print(f"  Title: {original_book.title}")
print(f"  ISBN: {original_book.isbn}")
print(f"  Author: {original_book.author}")
print(f"  Copies: {original_book.copies}")
print(f"  Object ID: {id(original_book)}")
print()

updated_book = replace(original_book, title="Effective Python (2nd Edition)")

print("Updated book (created with replace):")
print(f"  Title: {updated_book.title}")
print(f"  ISBN: {updated_book.isbn}")
print(f"  Author: {updated_book.author}")
print(f"  Copies: {updated_book.copies}")
print(f"  Object ID: {id(updated_book)}")
print()

print("Original book (unchanged):")
print(f"  Title: {original_book.title}")
print(f"  ISBN: {original_book.isbn}")
print(f"  Author: {original_book.author}")
print(f"  Copies: {original_book.copies}")
print(f"  Object ID: {id(original_book)}")
print()

print(f"Are they the same object? {original_book is updated_book}")
print(f"Are they equal (by ISBN)? {original_book == updated_book}")
print()

## 9) Rich string representations

Customize `__repr__` and `__str__` for `Book` (or confirm from scaffold) and explain in a short comment how they help during debugging vs. user display.

In [ ]:
# Your code here

@total_ordering
@dataclass(order=False)
class Book:
    isbn: str
    title: str
    _copies: int = field(default=1, repr=False, compare=False)
    author: str = field(default="Unknown")
    _title: str = field(default="", init=False, repr=False, compare=False)

    def __post_init__(self):
        self.isbn = self.isbn.replace("-", "").replace(" ", "")
        if not self.isbn or len(self.isbn) < 10:
            raise ValueError("ISBN must be at least 10 characters after normalization")
        self.title = self.title
        self._title = self._title.upper()

    def __repr__(self) -> str:
        return f"Book(isbn={self.isbn!r}, title={self._title!r}, author={self.author!r}, copies={self._copies})"

    def __str__(self) -> str:
        return f"{self._title} by {self.author} [{self.isbn}]"


    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return self.isbn == other.isbn

    def __lt__(self, other: "Book") -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return (self._title, self.isbn) < (other._title, other.isbn)

    def __hash__(self) -> int:
        return hash(self.isbn)

    @property
    def copies(self) -> int:
        return self._copies

    @copies.setter
    def copies(self, value: int) -> None:
        if not isinstance(value, int):
            raise TypeError("copies must be int")
        if value < 0:
            raise ValueError("copies must be non-negative")
        self._copies = value

    @property
    def title(self) -> str:
        return self._title

    @title.setter
    def title(self, value: str) -> None:
        if not isinstance(value, str):
            raise TypeError("title must be a string")
        stripped = value.strip()
        if not stripped:
            raise ValueError("Title cannot be empty after stripping whitespace")
        self._title = stripped

    @property
    def label(self) -> str:
        return f"{self.title} [{self.isbn}]"

## 10) Truthiness protocol

Define `__bool__(self)` on `Book` so that a book evaluates to `True` if `copies > 0` and `False` otherwise. Show a one-line `if book:` demo.

In [ ]:
# Your code here

@total_ordering
@dataclass(order=False)
class Book:
    isbn: str
    title: str
    _copies: int = field(default=1, repr=False, compare=False)
    author: str = field(default="Unknown")
    _title: str = field(default="", init=False, repr=False, compare=False)

    def __post_init__(self):
        self.isbn = self.isbn.replace("-", "").replace(" ", "")
        if not self.isbn or len(self.isbn) < 10:
            raise ValueError("ISBN must be at least 10 characters after normalization")
        self.title = self.title
        self._title = self._title.upper()

    def __bool__(self) -> bool:
        return self._copies > 0

    def __repr__(self) -> str:
        return f"Book(isbn={self.isbn!r}, title={self._title!r}, author={self.author!r}, copies={self._copies})"

    def __str__(self) -> str:
        return f"{self._title} by {self.author} [{self.isbn}]"

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return self.isbn == other.isbn

    def __lt__(self, other: "Book") -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return (self._title, self.isbn) < (other._title, other.isbn)

    def __hash__(self) -> int:
        return hash(self.isbn)

    @property
    def copies(self) -> int:
        return self._copies

    @copies.setter
    def copies(self, value: int) -> None:
        if not isinstance(value, int):
            raise TypeError("copies must be int")
        if value < 0:
            raise ValueError("copies must be non-negative")
        self._copies = value

    @property
    def title(self) -> str:
        return self._title

    @title.setter
    def title(self, value: str) -> None:
        if not isinstance(value, str):
            raise TypeError("title must be a string")
        stripped = value.strip()
        if not stripped:
            raise ValueError("Title cannot be empty after stripping whitespace")
        self._title = stripped

    @property
    def is_available(self) -> bool:
        return self._copies > 0

    @property
    def label(self) -> str:
        return f"{self.title} [{self.isbn}]"


## 11) Hashability decision

Decide whether `Book` should be hashable based on ISBN. If yes, implement `__hash__` consistent with `__eq__`. If not, explain why and show how using `Book` as a `dict` key could be risky if mutable fields affect identity.

In [ ]:
# Your code here

@total_ordering
@dataclass(order=False)
class Book:
    isbn: str
    title: str
    _copies: int = field(default=1, repr=False, compare=False)
    author: str = field(default="Unknown")
    _title: str = field(default="", init=False, repr=False, compare=False)

    def __post_init__(self):
        self.isbn = self.isbn.replace("-", "").replace(" ", "")
        if not self.isbn or len(self.isbn) < 10:
            raise ValueError("ISBN must be at least 10 characters after normalization")
        self.title = self.title
        self._title = self._title.upper()

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return self.isbn == other.isbn

    def __hash__(self) -> int:
        return hash(self.isbn)

    def __bool__(self) -> bool:
        return self._copies > 0

    def __lt__(self, other: "Book") -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return (self._title, self.isbn) < (other._title, other.isbn)

    def __repr__(self) -> str:
        return f"Book(isbn={self.isbn!r}, title={self._title!r}, author={self.author!r}, copies={self._copies})"

    def __str__(self) -> str:
        return f"{self._title} by {self.author} [{self.isbn}]"

    @property
    def copies(self) -> int:
        return self._copies

    @copies.setter
    def copies(self, value: int) -> None:
        if not isinstance(value, int):
            raise TypeError("copies must be int")
        if value < 0:
            raise ValueError("copies must be non-negative")
        self._copies = value

    @property
    def title(self) -> str:
        return self._title

    @title.setter
    def title(self, value: str) -> None:
        if not isinstance(value, str):
            raise TypeError("title must be a string")
        stripped = value.strip()
        if not stripped:
            raise ValueError("Title cannot be empty after stripping whitespace")
        self._title = stripped

## 12) In-place operations vs. new objects

Write a method `with_more_copies(self, n: int) -> Book` that returns a **new** `Book` with copies increased by `n` (do not mutate `self`). Show a quick before/after demonstration.

In [ ]:
# Your code here

from dataclasses import replace

@total_ordering
@dataclass(order=False)
class Book:
    isbn: str
    title: str
    _copies: int = field(default=1, repr=False, compare=False)
    author: str = field(default="Unknown")
    _title: str = field(default="", init=False, repr=False, compare=False)

    def __post_init__(self):
        self.isbn = self.isbn.replace("-", "").replace(" ", "")
        if not self.isbn or len(self.isbn) < 10:
            raise ValueError("ISBN must be at least 10 characters after normalization")
        self.title = self.title
        self._title = self._title.upper()

    def with_more_copies(self, n: int) -> "Book":
        new_copies = self._copies + n
        if new_copies < 0:
            raise ValueError(f"Cannot have negative copies (would be {new_copies})")
        return replace(self, _copies=new_copies)

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return self.isbn == other.isbn

    def __hash__(self) -> int:
        return hash(self.isbn)

    def __bool__(self) -> bool:
        return self._copies > 0

    def __lt__(self, other: "Book") -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return (self._title, self.isbn) < (other._title, other.isbn)

    def __repr__(self) -> str:
        return f"Book(isbn={self.isbn!r}, title={self._title!r}, author={self.author!r}, copies={self._copies})"

    def __str__(self) -> str:
        return f"{self._title} by {self.author} [{self.isbn}]"

    @property
    def copies(self) -> int:
        return self._copies

    @copies.setter
    def copies(self, value: int) -> None:
        if not isinstance(value, int):
            raise TypeError("copies must be int")
        if value < 0:
            raise ValueError("copies must be non-negative")
        self._copies = value

    @property
    def title(self) -> str:
        return self._title

    @title.setter
    def title(self, value: str) -> None:
        if not isinstance(value, str):
            raise TypeError("title must be a string")
        stripped = value.strip()
        if not stripped:
            raise ValueError("Title cannot be empty after stripping whitespace")
        self._title = stripped

    @property
    def is_available(self) -> bool:
        return self._copies > 0

## 13) Catalog iteration contract

Demonstrate that the provided `Catalog.__iter__` yields books in **sorted** order by title. Add an assertion-based test in a code cell.

In [ ]:
# Your code here
catalog = Catalog()
books_to_add = [
    Book(isbn="978-1491946008", title="Fluent Python", author="Luciano Ramalho", _copies=5),
    Book(isbn="978-0134685991", title="Effective Python", author="Brett Slatkin", _copies=3),
    Book(isbn="978-0132350884", title="Clean Code", author="Robert Martin", _copies=4),
    Book(isbn="978-0135957059", title="A Philosophy of Software Design", author="John Ousterhout", _copies=2),
    Book(isbn="978-0201633610", title="Design Patterns", author="Gang of Four", _copies=1),
    Book(isbn="978-0596007126", title="Beautiful Code", author="Various", _copies=2),
]

print("=" * 70)
print("DEMONSTRATION: Catalog.__iter__ yields books in sorted order")
print("=" * 70)
print()

print("Adding books in RANDOM order:")
for i, book in enumerate(books_to_add, 1):
    catalog.add_book(book)
    print(f"  {i}. {book.title}")
print()

print("Iterating over catalog (should be SORTED by title):")
iteration_order = []
for i, book in enumerate(catalog, 1):
    print(f"  {i}. {book.title}")
    iteration_order.append(book.title)
print()

print("=" * 70)
print("ASSERTION TEST:")
print("=" * 70)
print()

expected_order = [
    "A PHILOSOPHY OF SOFTWARE DESIGN",
    "BEAUTIFUL CODE",
    "CLEAN CODE",
    "DESIGN PATTERNS",
    "EFFECTIVE PYTHON",
    "FLUENT PYTHON",
]

assert iteration_order == expected_order, f"Expected {expected_order}, got {iteration_order}"
print("✓ TEST PASSED: Books are yielded in alphabetically sorted order by title")
print()

print("Summary:")
print(f"  - Added {len(books_to_add)} books in random order")
print(f"  - Iteration yielded all {len(iteration_order)} books in sorted order")
print(f"  - Catalog.__iter__ uses: iter(sorted(self._books.values()))")
print(f"  - Sorting is by Book.__lt__ which compares (title, isbn)")

## 14) Membership semantics

Show that `isbn in catalog` and `book in catalog` both work via `__contains__`. Add a short test cell to verify both cases.

In [ ]:
# Your code here
catalog = Catalog()

book1 = Book(isbn="978-0134685991", title="Effective Python", author="Brett Slatkin", _copies=5)
book2 = Book(isbn="978-1491946008", title="Fluent Python", author="Luciano Ramalho", _copies=3)
book3 = Book(isbn="978-0132350884", title="Clean Code", author="Robert Martin", _copies=4)

catalog.add_book(book1)
catalog.add_book(book2)

print("=" * 70)
print("DEMONSTRATION: Catalog.__contains__ supports ISBN and Book objects")
print("=" * 70)
print()

print("Books in catalog:")
for book in catalog:
    print(f"  - {book.title} [{book.isbn}]")
print()

print("Testing __contains__ with ISBN strings:")
print(f"  '9780134685991' in catalog: {('9780134685991' in catalog)}")
print(f"  '9781491946008' in catalog: {('9781491946008' in catalog)}")
print(f"  '9780132350884' in catalog: {('9780132350884' in catalog)} (not added)")
print()

print("Testing __contains__ with Book objects:")
print(f"  book1 in catalog: {(book1 in catalog)}")
print(f"  book2 in catalog: {(book2 in catalog)}")
print(f"  book3 in catalog: {(book3 in catalog)} (not added)")
print()

book1_copy = Book(isbn="978-0134685991", title="Effective Python", _copies=10)
print("Testing with a different Book object but same ISBN:")
print(f"  book1_copy (same ISBN as book1) in catalog: {(book1_copy in catalog)}")
print()

print("=" * 70)
print("ASSERTION TESTS:")
print("=" * 70)
print()

# Test 1: ISBN string lookup (in catalog)
assert "9780134685991" in catalog, "ISBN '9780134685991' should be in catalog"
print(" Test 1 PASSED: ISBN string '9780134685991' found in catalog")

# Test 2: ISBN string lookup (not in catalog)
assert "9780132350884" not in catalog, "ISBN '9780132350884' should NOT be in catalog"
print("Test 2 PASSED: ISBN string '9780132350884' correctly not found")

## 15) Defensive copying in accessors

Add a method `to_list(self) -> list[Book]` to `Catalog` that returns a shallow copy of the book list (sorted). Explain why returning internal structures directly is risky.

In [ ]:
# Your code here
class Catalog(Iterable[Book]):
    def __init__(self):
        self._books: Dict[str, Book] = {}
    
    def __len__(self) -> int:
        return len(self._books)

    def __iter__(self) -> Iterator[Book]:
        return iter(sorted(self._books.values()))

    def __contains__(self, isbn: object) -> bool:
        if isinstance(isbn, Book):
            return isbn.isbn in self._books
        return isinstance(isbn, str) and (isbn in self._books)

    def __getitem__(self, isbn: str) -> Book:
        return self._books[isbn]

    def __delitem__(self, isbn: str) -> None:
        if isbn not in self._books:
            raise KeyError(f"Book with ISBN {isbn} not found in catalog")
        del self._books[isbn]

    def __str__(self) -> str:
        return f"Catalog with {len(self)} book(s)"

    def __repr__(self) -> str:
        return f"Catalog({list(self._books.keys())})"

  
    def add_book(self, book: Book) -> None:
        if book.isbn in self._books:
            raise DuplicateBookError(f"ISBN exists: {book.isbn}")
        self._books[book.isbn] = book

    def get_book(self, isbn: str) -> Optional[Book]:
        return self._books.get(isbn)

    def remove_book(self, isbn: str) -> Optional[Book]:
        return self._books.pop(isbn, None)

    def search_by_title(self, title: str) -> List[Book]:
        title_lower = title.lower()
        return [b for b in self if title_lower in b._title.lower()]

    def find_by_title_prefix(self, prefix: str) -> List[Book]:
        prefix_lower = prefix.lower()
        return [book for book in self if book._title.lower().startswith(prefix_lower)]

    def to_list(self) -> List[Book]:
        return list(self) 

## 16) Lightweight value object

Create a small `@dataclass(frozen=True)` called `Author` with fields `last`, `first`. Add an optional `author: Author | None` to `Book` (default `None`). Show how `frozen=True` prevents later mutation and why that can be desirable for identity keys.

In [ ]:
# Your code here

from dataclasses import dataclass, field
from typing import Optional

@dataclass(frozen=True)
class Author:
    last: str
    first: str
    
    def __str__(self) -> str:
        return f"{self.first} {self.last}"


@total_ordering
@dataclass(order=False)
class Book:
    isbn: str
    title: str
    _copies: int = field(default=1, repr=False, compare=False)
    author: Optional[Author] = None  # New field: optional Author object
    _title: str = field(default="", init=False, repr=False, compare=False)

    def __post_init__(self):
        self.isbn = self.isbn.replace("-", "").replace(" ", "")
        if not self.isbn or len(self.isbn) < 10:
            raise ValueError("ISBN must be at least 10 characters after normalization")
        self.title = self.title
        self._title = self._title.upper()

    def __bool__(self) -> bool:
        return self._copies > 0

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return self.isbn == other.isbn

    def __hash__(self) -> int:
        return hash(self.isbn)

    def __lt__(self, other: "Book") -> bool:
        if not isinstance(other, Book):
            return NotImplemented
        return (self._title, self.isbn) < (other._title, other.isbn)

    def __repr__(self) -> str:
        author_repr = f"author={self.author!r}" if self.author else "author=None"
        return f"Book(isbn={self.isbn!r}, title={self._title!r}, {author_repr}, copies={self._copies})"

    def __str__(self) -> str:
        if self.author:
            return f"{self._title} by {self.author} [{self.isbn}]"
        return f"{self._title} [author unknown] [{self.isbn}]"

    @property
    def copies(self) -> int:
        return self._copies

    @copies.setter
    def copies(self, value: int) -> None:
        if not isinstance(value, int):
            raise TypeError("copies must be int")
        if value < 0:
            raise ValueError("copies must be non-negative")
        self._copies = value

    @property
    def title(self) -> str:
        return self._title

    @title.setter
    def title(self, value: str) -> None:
        if not isinstance(value, str):
            raise TypeError("title must be a string")
        stripped = value.strip()
        if not stripped:
            raise ValueError("Title cannot be empty after stripping whitespace")
        self._title = stripped



## 17) Custom containment by predicate

Implement `contains_title(self, needle: str) -> bool` on `Catalog` that returns `True` if any book title equals `needle` case-insensitively. Do not modify `__contains__`.

In [ ]:
# Your code here

class Catalog(Iterable[Book]):
    def __init__(self):
        self._books: Dict[str, Book] = {}

    def __len__(self) -> int:
        return len(self._books)

    def __iter__(self) -> Iterator[Book]:
        return iter(sorted(self._books.values()))

    def __contains__(self, isbn: object) -> bool:
        if isinstance(isbn, Book):
            return isbn.isbn in self._books
        return isinstance(isbn, str) and (isbn in self._books)

    def __getitem__(self, isbn: str) -> Book:
        return self._books[isbn]

    def __delitem__(self, isbn: str) -> None:
        if isbn not in self._books:
            raise KeyError(f"Book with ISBN {isbn} not found in catalog")
        del self._books[isbn]

    def __str__(self) -> str:
        return f"Catalog with {len(self)} book(s)"

    def __repr__(self) -> str:
        return f"Catalog({list(self._books.keys())})"

    def add_book(self, book: Book) -> None:
        if book.isbn in self._books:
            raise DuplicateBookError(f"ISBN exists: {book.isbn}")
        self._books[book.isbn] = book

    def get_book(self, isbn: str) -> Optional[Book]:
        return self._books.get(isbn)

    def remove_book(self, isbn: str) -> Optional[Book]:
        return self._books.pop(isbn, None)

    def search_by_title(self, title: str) -> List[Book]:
        title_lower = title.lower()
        return [b for b in self if title_lower in b._title.lower()]

    def find_by_title_prefix(self, prefix: str) -> List[Book]:
        prefix_lower = prefix.lower()
        return [book for book in self if book._title.lower().startswith(prefix_lower)]

    def to_list(self) -> List[Book]:
        return list(self)

    def contains_title(self, needle: str) -> bool:
        needle_lower = needle.lower()
        return any(book._title.lower() == needle_lower for book in self._books.values())


## 18) Pretty-print table helper

Write a standalone function `print_catalog_table(catalog: Catalog)` that prints a simple aligned table of ISBN, Title, Copies. Use only string formatting, no third-party libs.

In [ ]:
# Your code here
def print_catalog_table(catalog: Catalog) -> None:
    books = catalog.to_list()
    
    if not books:
        print("Catalog is empty - no books to display.")
        return
    
    isbn_width = max(len(book.isbn) for book in books)
    isbn_width = max(isbn_width, len("ISBN"))  
    
    title_width = max(len(book._title) for book in books)
    title_width = max(title_width, len("TITLE"))
    
    copies_width = max(len(str(book.copies)) for book in books)
    copies_width = max(copies_width, len("COPIES"))
    
    separator = f"+{'-' * (isbn_width + 2)}+{'-' * (title_width + 2)}+{'-' * (copies_width + 2)}+"
    
    print(separator)

    header = f"| {'ISBN':<{isbn_width}} | {'TITLE':<{title_width}} | {'COPIES':>{copies_width}} |"
    print(header)
    print(separator)
   
    for book in books:
        row = f"| {book.isbn:<{isbn_width}} | {book._title:<{title_width}} | {book.copies:>{copies_width}} |"
        print(row)
    
    print(separator)
    
    total_books = len(books)
    total_copies = sum(book.copies for book in books)
    print(f"Total: {total_books} book(s), {total_copies} copies")


## 19) Sorting with key functions

Demonstrate sorting books by different keys without changing their natural order: by ISBN, by copies (descending), by title length. Use `sorted(..., key=...)`.

In [ ]:
# Your code here


print("=" * 70)
print("DEMONSTRATION: Sorting books by different keys")
print("=" * 70)
print()

catalog = Catalog()

books = [
    Book(isbn="978-0134685991", title="Effective Python", _copies=5),
    Book(isbn="978-1491946008", title="Fluent Python", _copies=3),
    Book(isbn="978-0132350884", title="Clean Code", _copies=12),
    Book(isbn="978-0596007126", title="Beautiful Code", _copies=2),
    Book(isbn="978-0135957059", title="A Philosophy of Software Design", _copies=7),
    Book(isbn="978-0201633610", title="Design Patterns", _copies=1),
    Book(isbn="978-0321563842", title="Effective Java", _copies=8),
]

for book in books:
    catalog.add_book(book)

print("Original catalog (natural order - by title via __iter__):")
for i, book in enumerate(catalog, 1):
    print(f"  {i}. {book._title} [{book.isbn}] - {book.copies} copies")
print()

print("=" * 70)
print("SORT 1: By ISBN (ascending)")
print("=" * 70)
print()

sorted_by_isbn = sorted(catalog, key=lambda book: book.isbn)

for i, book in enumerate(sorted_by_isbn, 1):
    print(f"  {i}. ISBN: {book.isbn} - {book._title} - {book.copies} copies")
print()

print("=" * 70)
print("SORT 2: By copies (descending - most copies first)")
print("=" * 70)
print()

sorted_by_copies_desc = sorted(catalog, key=lambda book: book.copies, reverse=True)

for i, book in enumerate(sorted_by_copies_desc, 1):
    print(f"  {i}. {book.copies:>3} copies - {book._title}")
print()

print("=" * 70)
print("SORT 3: By title length (shortest first)")
print("=" * 70)
print()

sorted_by_title_length = sorted(catalog, key=lambda book: len(book._title))

for i, book in enumerate(sorted_by_title_length, 1):
    print(f"  {i}. Length {len(book._title):>2}: {book._title}")
print()

print("=" * 70)
print("SORT 4: By title length (longest first)")
print("=" * 70)
print()

sorted_by_title_length_desc = sorted(catalog, key=lambda book: len(book._title), reverse=True)

for i, book in enumerate(sorted_by_title_length_desc, 1):
    print(f"  {i}. Length {len(book._title):>2}: {book._title}")
print()

print("=" * 70)
print("SORT 5: By availability (available books first, then by title)")
print("=" * 70)
print()


catalog.add_book(Book(isbn="978-0000000000", title="Out of Stock Book", _copies=0))

sorted_by_availability = sorted(
    catalog, 
    key=lambda book: (not book.is_available, book._title)
)

for i, book in enumerate(sorted_by_availability, 1):
    status = "✓ Available" if book.is_available else "✗ Out of Stock"
    print(f"  {i}. {status:15} - {book._title} ({book.copies} copies)")
print()

print("=" * 70)
print("SORT 6: By author last name (with None handling)")
print("=" * 70)
print()


catalog_with_authors = Catalog()

author1 = Author(last="Slatkin", first="Brett")
author2 = Author(last="Ramalho", first="Luciano")
author3 = Author(last="Martin", first="Robert")
author4 = Author(last="Bloch", first="Joshua")

catalog_with_authors.add_book(Book(isbn="978-0134685991", title="Effective Python", author=author1, _copies=5))
catalog_with_authors.add_book(Book(isbn="978-1491946008", title="Fluent Python", author=author2, _copies=3))
catalog_with_authors.add_book(Book(isbn="978-0132350884", title="Clean Code", author=author3, _copies=12))
catalog_with_authors.add_book(Book(isbn="978-0321563842", title="Effective Java", author=author4, _copies=8))
catalog_with_authors.add_book(Book(isbn="978-0596007126", title="Beautiful Code", author=None, _copies=2))


sorted_by_author = sorted(
    catalog_with_authors,
    key=lambda book: (book.author is None, book.author.last if book.author else "")
)

for i, book in enumerate(sorted_by_author, 1):
    author_name = str(book.author) if book.author else "Unknown"
    print(f"  {i}. {author_name:20} - {book._title}")
print()

print("=" * 70)
print("SORT 7: Complex multi-level sort (copies desc, then title length asc)")
print("=" * 70)
print()


sorted_complex = sorted(
    catalog,
    key=lambda book: (-book.copies, len(book._title))
)

for i, book in enumerate(sorted_complex, 1):
    print(f"  {i}. {book.copies:>3} copies, length {len(book._title):>2}: {book._title}")
print()

print("=" * 70)
print("SORT 8: By first word of title")
print("=" * 70)
print()

sorted_by_first_word = sorted(
    catalog,
    key=lambda book: book._title.split()[0]
)

for i, book in enumerate(sorted_by_first_word, 1):
    first_word = book._title.split()[0]
    print(f"  {i}. First word '{first_word}': {book._title}")
print()

print("=" * 70)
print("Verification: Natural order unchanged")
print("=" * 70)
print()

print("After all those sorts, the catalog's natural order is still by title:")
for i, book in enumerate(catalog, 1):
    print(f"  {i}. {book._title}")
print()

print("This is because sorted() creates NEW lists without modifying the original!")
print()



## 20) Minimal unit tests for data model

Using `unittest`, write a small test case verifying:
- `Book.title` setter strips and forbids empty
- `Book.copies` setter validates
- `Catalog` iteration is sorted
- `__contains__` works for both ISBN and Book
(Keep it basic—no fixtures beyond a simple `setUp`.)

In [ ]:
# Your code here
import unittest

class TestWeek10DataModel(unittest.TestCase):
    def setUp(self):
        self.c = Catalog()
        self.c.add_book(Book("2","Beta",1))
        self.c.add_book(Book("1","Alpha",2))
    def test_title_property(self):
        b = Book("3","  New  ",1)
        # Test 1: Title is stripped on creation
        b = Book("3333333333", "  New  ", _copies=1)
        self.assertEqual(b.title, "NEW")  
        
        # Test 2: Title is stripped on setter
        b.title = "  Updated Title  "
        self.assertEqual(b.title, "UPDATED TITLE")
        
        # Test 3: Empty title after stripping raises ValueError
        with self.assertRaises(ValueError):
            b.title = "   "  
        
        # Test 4: Empty string raises ValueError
        with self.assertRaises(ValueError):
            b.title = ""
        
        # Test 5: Original valid title is preserved after failed set
        self.assertEqual(b.title, "UPDATED TITLE")
    
    def test_copies_validation(self):
        b = Book("4","Copies",1)
        b = Book("4444444444", "Copies", _copies=1)
        
        # Test 1: Valid positive value works
        b.copies = 10
        self.assertEqual(b.copies, 10)
        
        # Test 2: Zero is valid
        b.copies = 0
        self.assertEqual(b.copies, 0)
        
        # Test 3: Negative value raises ValueError
        with self.assertRaises(ValueError):
            b.copies = -1
        
        # Test 4: Non-integer raises TypeError
        with self.assertRaises(TypeError):
            b.copies = "5"
        
        # Test 5: Float raises TypeError
        with self.assertRaises(TypeError):
            b.copies = 3.14
        
        # Test 6: Value preserved after failed validation
        self.assertEqual(b.copies, 0)
    def test_sorted_iteration(self):
        titles = [b.title for b in self.c]
        titles = [b.title for b in self.c]
        
        # Test 1: Should be sorted alphabetically
        self.assertEqual(titles, ["ALPHA", "BETA"])
        
        # Test 2: Should match sorted version
        self.assertEqual(titles, sorted(titles))
        
        # Test 3: Add more books and verify still sorted
        self.c.add_book(Book("5555555555", "Zeta", _copies=3))
        self.c.add_book(Book("6666666666", "Delta", _copies=4))
        
        titles_after = [b.title for b in self.c]
        self.assertEqual(titles_after, ["ALPHA", "BETA", "DELTA", "ZETA"])
        self.assertEqual(titles_after, sorted(titles_after))
    def test_contains(self):
        a = self.c.get_book("1")
        a = self.c.get_book("1111111111")
        
        # Test 1: ISBN string lookup (in catalog)
        self.assertIn("1111111111", self.c)
        self.assertIn("2222222222", self.c)
        
        # Test 2: ISBN string lookup (not in catalog)
        self.assertNotIn("9999999999", self.c)
        self.assertNotIn("0000000000", self.c)
        
        # Test 3: Book object lookup (in catalog)
        self.assertIn(a, self.c)
        
        # Test 4: Book object with same ISBN (should be found)
        a_copy = Book("1111111111", "Alpha Copy", _copies=999)
        self.assertIn(a_copy, self.c)  # Same ISBN, so found
        
        # Test 5: Book object not in catalog
        b_new = Book("7777777777", "New Book", _copies=1)
        self.assertNotIn(b_new, self.c)
        
        # Test 6: After adding, book should be found
        self.c.add_book(b_new)
        self.assertIn(b_new, self.c)
        self.assertIn("7777777777", self.c)

# # To run in notebook:
# # unittest.main(argv=['-v'], exit=False)

## Python skills you'll need (Weeks 1–10)

- **Core syntax & data types:** variables, strings, numbers, booleans
- **Collections:** lists, dicts; simple comprehensions
- **Control flow:** `if/elif/else`, `for`, `while`
- **Functions & modules:** defining functions, parameters, returns, imports
- **File I/O & JSON (basic):** open/read/write
- **Testing (Week 7):** `unittest.TestCase`, assertions, simple setup
- **OOP (Weeks 4–9):** classes, methods, inheritance (single), ABCs (light), polymorphism
- **Week 10 focus (data model):**
  - `@property` getters/setters/deleters
  - `dataclass` options: `field`, `__post_init__`, equality & ordering
  - Special methods: `__str__`, `__repr__`, `__len__`, `__iter__`, `__contains__`, `__eq__`, ordering (`total_ordering`)
  - Lightweight containers and defensive copying
  - Optional: `dataclasses.replace`
- **Standard library familiarity:** `dataclasses`, `functools.total_ordering`, `datetime`
